### Import das Bibliotecas que serão usadas

In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## 0 - Definindo funcoes auxiliares

### 0.1 - Função para extrair valores associados a uma chave no formato JSON

##### 0.1.1 - Função para extrair nome dos gêneros, keywords e atores do filme

In [2]:
def converter_atributos(objeto):
    lista_aux =[]
    for i in ast.literal_eval(objeto):
        lista_aux.append(i['name'])
    return lista_aux

##### 0.1.2 - Função para extrair o nome do diretor

In [3]:
def converter_diretor(objeto):
    lista_aux =[]
    for i in ast.literal_eval(objeto):
        if(i['job']=='Director'):
            lista_aux.append(i['name'])
            break
    return lista_aux

## 1 - Criando o dataframe

### 1.1 - Fazendo a Leitura dos dados

In [4]:
filmes_df = pd.read_csv('movies.csv')
credits_df = pd.read_csv('credits.csv')

In [5]:
#filmes_df.head()

### 1.2 - Juntando os dataframes em um unico

In [6]:
filmes_df = filmes_df.merge(credits_df,on='title')

In [7]:
#filmes_df.head()

### 1.3 - Selecionando apenas os atributos que serão usados

In [8]:
filmes_df = filmes_df[['movie_id','title','genres','keywords','overview','cast','crew']]

In [9]:
#filmes_df.head()

## 2 - Arrumando o dataframe

### 2.1 - Removendo as linhas nulas

In [10]:
filmes_df.dropna(inplace=True)

In [11]:
#filmes_df.head()

### 2.2 - Extraindo os valores para os atributos correspondetes

In [12]:
filmes_df['genres'] = filmes_df['genres'].apply(converter_atributos)
filmes_df['keywords'] = filmes_df['keywords'].apply(converter_atributos)
filmes_df['cast'] = filmes_df['cast'].apply(converter_atributos)

In [13]:
#filmes_df.head()

In [14]:
filmes_df['crew'] = filmes_df['crew'].apply(converter_diretor)

In [15]:
#filmes_df.head()

### 2.3 - Manipulando os valores de cada atributo

##### 2.3.1 - Transformando a sinopse em uma lista de palavras

In [16]:
filmes_df['overview'] = filmes_df['overview'].apply(lambda x:x.split())

In [17]:
#filmes_df.head()

##### 2.3.2 - Removendo o espaço entre palavras

In [18]:
filmes_df['genres'] = filmes_df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
filmes_df['keywords'] = filmes_df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
filmes_df['overview'] = filmes_df['overview'].apply(lambda x:[i.replace(" ","") for i in x])
filmes_df['cast'] = filmes_df['cast'].apply(lambda x:[i.replace(" ","") for i in x])
filmes_df['crew'] = filmes_df['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [19]:
#filmes_df.head()

### 3 - Criando novo dataframe que será manipulado para recomendar um filme

In [20]:
filmes_aux_df = filmes_df[['movie_id','title']]
filmes_aux_df['tags'] = filmes_df['overview'] + filmes_df['genres'] + filmes_df['keywords'] + filmes_df['cast'] + filmes_df['crew']

/tmp/ipykernel_127680/3371148743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes_aux_df['tags'] = filmes_df['overview'] + filmes_df['genres'] + filmes_df['keywords'] + filmes_df['cast'] + filmes_df['crew']


In [21]:
#filmes_aux_df.head()

#### 3.1 - Transformando os atributos de tags em strings, não mais em uma lista

In [22]:
filmes_aux_df['tags'] = filmes_aux_df['tags'].apply(lambda x:" ".join(x))

/tmp/ipykernel_127680/3652816384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes_aux_df['tags'] = filmes_aux_df['tags'].apply(lambda x:" ".join(x))


In [23]:
#filmes_aux_df.head()

#### 3.2 - Convertendo as strings para letras minúsculas

In [24]:
filmes_aux_df['tags'] = filmes_aux_df['tags'].apply(lambda x:x.lower())

/tmp/ipykernel_127680/3700548260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmes_aux_df['tags'] = filmes_aux_df['tags'].apply(lambda x:x.lower())


In [25]:
#filmes_aux_df.head()

### 4 - Criando o sistema de recomendação

#### 4.1 - Criando a matriz de contagem de tokens

In [26]:
cv = CountVectorizer(max_features=5500,stop_words='english')

In [27]:
vetores = cv.fit_transform(filmes_aux_df['tags']).toarray()

In [28]:
#vectors[0][10:40]

In [29]:
#palavras unicas utilizadas


#cv.get_feature_names_out()

#### 4.2 - Método de aprendizado do modelo

In [30]:
matriz_similaridade = cosine_similarity(vetores)

In [31]:
#matriz_similaridade

#### 4.3 - Salvando os Dataframes em binários

In [32]:
pickle.dump(filmes_aux_df.to_dict(),open('dicionario_filmes.pkl','wb'))

In [33]:
pickle.dump(matriz_similaridade,open('similaridade.pkl','wb'))

In [34]:
pickle.dump(filmes_aux_df,open('movie_list.pkl','wb'))
